In [19]:
import cv2
import os
from facenet_pytorch import MTCNN
from PIL import Image

In [13]:
def file_path(fake_folder, original_folder):
    fake_videos = [fake_folder+'/'+ f for f in os.listdir(fake_folder) if f.endswith(('.mp4', '.avi'))]
    original_videos = [original_folder+'/'+ f for f in os.listdir(original_folder) if f.endswith(('.mp4', '.avi'))]
    
    video_paths = fake_videos + original_videos
    labels = [1] * len(fake_videos) + [0] * len(original_videos)

    return video_paths, labels

In [14]:
fake_folder = "archive/DFD_manipulated_sequences"
original_folder = "archive/DFD_original sequences"
video_paths, labels = file_path(fake_folder, original_folder)

In [ ]:
def extract_frames(video_path, sample_rate=10):

    """Extracting frames from a video at a given sample rate"""

    vid = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while vid.isOpened():
        ret, frame = vid.read()
        if not ret: 
            break
        
        if count % sample_rate == 0:
            frames.append(frame)
        
        count += 1
    
    vid.release()
    return frames

def crop_faces(frames):

    cropped_faces = []
    for frame in frames:
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        boxes, _ = mtcnn.detect(frame_rgb)
        if boxes:
            for box in boxes:
                x1, y1, x2, y2 = [int(coord) for coord in box]
                face = frame_rgb[y1:y2, x1:x2]
                cropped_faces.append(face)
    
    return cropped_faces


3431

In [20]:
mtcnn = MTCNN(image_size=224, margin=20)